In [1]:
import pandas as pd
import requests

# url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ADM%2C+BKL%2C+BMW%2C+BNK%2C+CDP%2C+DSS%2C+FOR%2C+GOL%2C+GRZ%2C+HMB%2C+IDC%2C+IDP%2C+INN%2C+LLP%2C+MED%2C+NLS%2C+PLP%2C+RRM%2C+RTL%2C+SHM%2C+SLT%2C++SNM%2C+SQV%2C+TK2%2C++ALP%2C+BLC%2C+BLK%2C+BLS%2C+BSK%2C+CAP%2C+CSL%2C+DAN%2C+DDM%2C+EBB%2C+EP5%2C+FDC%2C+FLL%2C+GIN%2C+GKS%2C+GNL%2C+HGM%2C+HOR%2C+HRS%2C+HVN%2C+HYS%2C+LBD%2C+LVM%2C+LVT%2C+MDW%2C+MNT%2C+MRL%2C+MSK%2C+PSN%2C+RBB%2C+RBP%2C+RP2%2C+SDW%2C+SIL%2C+SLI%2C+SPS%2C+SPT%2C+STR%2C+TCC%2C+TNY%2C+TUM%2C+VRG%2C+WC3%2C+WHW%2C+BCH%2C+BGP%2C+CHM%2C+CHP%2C+CRL%2C+CSV%2C+CWD%2C+DPO%2C+GNF%2C+GRM%2C+GRV%2C+KSP%2C+MTM%2C+PSC%2C+PSR%2C+RCK%2C+SLK%2C+STL%2C+SWM%2C+TMR%2C+TUN%2C+UTY%2C+VLC%2C+WTM&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-11"
url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ADM%2CBKL%2CBMW%2CBNK%2CCDP%2CDSS%2CFOR%2CGOL%2CGRZ%2CHMB%2CIDC%2CIDP%2CINN%2CLLP%2CMED%2CNLS%2CPLP%2CRRM%2CRTL%2CSHM%2CSLT%2CSNM%2CSQV%2CTK2%2C+ALP%2CBLC%2CBLK%2CBLS%2CBSK%2CCAP%2CCSL%2CDAN%2CDDM%2CEBB%2CEP5%2CFDC%2CFLL%2CGIN%2CGKS%2CGNL%2CHGM%2CHOR%2CHRS%2CHVN%2CHYS%2CLBD%2CLVM%2CLVT%2CMDW%2CMNT%2CMRL%2CMSK%2CPSN%2CRBB%2CRBP%2CRP2%2CSDW%2CSIL%2CSLI%2CSPS%2CSPT%2CSTR%2CTCC%2CTNY%2CTUM%2CVRG%2CWC3%2CWHW%2CBCH%2CBGP%2CCHM%2CCHP%2CCRL%2CCSV%2CCWD%2CDPO%2CGNF%2CGRM%2CGRV%2CKSP%2CMTM%2CPSC%2CPSR%2CRCK%2CSLK%2CSTL%2CSWM%2CTMR%2CTUN%2CUTY%2CVLC%2CWTM&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-13"
response = requests.get(url).json()

In [2]:
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year

array(['ADM', 'BNK', 'IDC', 'IDP', 'INN', 'LLP', 'MED', 'NLS', 'RRM',
       'SHM', 'SLT', 'SNM', 'SQV', 'TK2', 'BLC', 'BLK', 'BLS', 'BSK',
       'CAP', 'DAN', 'DDM', 'EBB', 'EP5', 'FDC', 'FLL', 'GIN', 'GKS',
       'GNL', 'HGM', 'HOR', 'HRS', 'HVN', 'HYS', 'LVM', 'LVT', 'MDW',
       'MNT', 'MRL', 'MSK', 'PSN', 'RP2', 'SDW', 'SIL', 'SLI', 'SPS',
       'SPT', 'STR', 'TCC', 'TNY', 'TUM', 'VRG', 'WC3', 'WHW', 'BCH',
       'BGP', 'CHM', 'CHP', 'CRL', 'CSV', 'CWD', 'DPO', 'GRM', 'GRV',
       'KSP', 'MTM', 'PSC', 'PSR', 'RCK', 'SLK', 'STL', 'SWM', 'TMR',
       'TUN', 'UTY', 'VLC', 'WTM'], dtype=object)

In [9]:
snow_df["Snow Depth (in)"] = snow_df["Snow Depth (in)"].clip(lower = 0)


0

In [11]:
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2014

for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year - 1} {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1

snow_season["Season"] = season_list

In [14]:
winter_only_mask = snow_season["Season"].str.contains("Winter")
snow_season_winter_only = snow_season.loc[winter_only_mask]
snow_season_grouped = snow_season_winter_only.groupby(["Season"])
snow_season_grouped.max().head(10)

,Date,Station ID,Snow Depth (in),Month,Year
Season,,,,,
Winter 2013 2014,2014-04-30,WHW,295,12,2014
Winter 2014 2015,2015-04-30,WHW,313,4,2015
Winter 2015 2016,2016-04-30,WHW,311,4,2016
Winter 2016 2017,2017-04-30,WHW,312,4,2017
Winter 2017 2018,2018-04-30,WHW,279,4,2018
Winter 2018 2019,2019-04-30,WTM,2317,4,2019
Winter 2019 2020,2020-04-30,WTM,332,4,2020
Winter 2020 2021,2021-04-30,WTM,322,4,2021
Winter 2021 2022,2022-04-30,WTM,329,4,2022
